In [9]:
import os
import sys
from pathlib import Path

mod_path = os.path.join(Path.cwd().parent.parent)
if mod_path not in sys.path:
    sys.path.append(mod_path)

In [10]:
import pandas as pd
import numpy as np

import xgboost as xgb
from sklearn.model_selection import StratifiedKFold

from src.data.pre_process import one_hot_encoding
from src.model.tree_based import ModelXgBoost

In [20]:
# data with shape 28007, 33 [transaction related features]
train = pd.read_csv('../../data/processed/train.csv')

In [21]:
train

,Unnamed: 0,RegistrationDate,Deposit,AccessoryRate,rateTypeEntity,RatePerUnit,DaysOnDeposit,MainApplicantGender,Age,Region,...,SplitPaymentsHistory,SplitTransactionDates,nb_payments,amount_paid,percent_amt_paid,mean_amt_paid,median_amt_paid,max_amt_paid,min_amt_paid,stddev_amt_paid
0,0,2018-04-23 12:28:05,2500,0.0,DAILY,55,3,Male,NaN,Coast Region,...,"['3600.0', ' 750.0', ' 350.0', ' 65.0', ' 95.0...","[""'04-2018'"", "" '05-2018'"", "" '06-2018'"", "" '0...",31,16035.0,0.484734,517.26,350.0,3600.0,55.0,652.49
1,1,2018-04-17 10:27:35,2500,0.0,DAILY,55,3,Male,26.0,South Rift,...,"['2940.0', ' 970.0', ' 380.0', ' 880.0', ' 385...","[""'04-2018'"", "" '05-2018'"", "" '06-2018'"", "" '0...",30,22136.0,0.669166,737.87,655.0,2940.0,380.0,452.03
2,2,2020-02-18 14:23:44,2400,0.0,DAILY,50,3,Male,21.0,Mount Kenya Region,...,"['2850.0', ' 1500.0', ' 1350.0', ' 610.0', ' 2...","[""'02-2020'"", "" '03-2020'"", "" '04-2020'"", "" '0...",6,6760.0,0.469444,1126.67,980.0,2850.0,200.0,1005.32
3,3,2017-09-14 11:07:40,2000,0.0,DAILY,40,7,Female,26.0,Mount Kenya Region,...,"['2200.0', ' 1420.0', ' 1180.0', ' 900.0', ' 1...","[""'09-2017'"", "" '10-2017'"", "" '11-2017'"", "" '1...",10,11260.0,0.679952,1126.00,1140.0,2200.0,380.0,511.30
4,4,2017-09-06 06:50:20,2000,0.0,DAILY,40,7,Male,27.0,North Rift,...,"['2640.0', ' 910.0', ' 480.0', ' 280.0', ' 200...","[""'09-2017'"", "" '10-2017'"", "" '11-2017'"", "" '1...",22,7779.0,0.469746,353.59,190.0,2640.0,40.0,559.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28002,28002,2018-03-24 09:52:09,2500,0.0,DAILY,55,3,Male,NaN,Coast Region,...,"['5235.0', ' 1270.0', ' 940.0', ' 1370.0', ' 2...","[""'03-2018'"", "" '04-2018'"", "" '05-2018'"", "" '0...",14,24545.0,0.741989,1753.21,1635.0,5235.0,800.0,1069.37
28003,28003,2020-02-17 13:34:57,2400,0.0,DAILY,50,3,Female,NaN,Nairobi Region,...,"['2980.0', ' 1000.0', ' 450.0', ' 1050.0', ' 8...","[""'02-2020'"", "" '03-2020'"", "" '04-2020'"", "" '0...",8,9330.0,0.647917,1166.25,1025.0,2980.0,450.0,786.66
28004,28004,2020-01-15 11:12:06,2000,0.0,DAILY,40,3,Male,29.0,South Rift,...,"['2580.0', ' 1020.0', ' 540.0', ' 360.0', ' 20...","[""'01-2020'"", "" '02-2020'"", "" '03-2020'"", "" '0...",9,8570.0,0.669531,952.22,1020.0,2580.0,200.0,695.70
28005,28005,2018-02-25 16:51:21,2000,0.0,DAILY,40,7,Female,57.0,Mount Kenya Region,...,"['2000.0', ' 1160.0', ' 1240.0', ' 1120.0', ' ...","[""'02-2018'"", "" '03-2018'"", "" '04-2018'"", "" '0...",8,9930.0,0.599638,1241.25,1180.0,2000.0,970.0,323.04


In [22]:
train.drop(columns = 'Unnamed: 0', inplace=True)
train.columns

Index(['RegistrationDate', 'Deposit', 'AccessoryRate', 'rateTypeEntity',
       'RatePerUnit', 'DaysOnDeposit', 'MainApplicantGender', 'Age', 'Region',
       'Town', 'Occupation', 'Term', 'TotalContractValue', 'ExpectedTermDate',
       'FirstPaymentDate', 'LastPaymentDate', 'm1', 'm2', 'm3', 'm4', 'm5',
       'm6', 'SplitPaymentsHistory', 'SplitTransactionDates', 'nb_payments',
       'amount_paid', 'percent_amt_paid', 'mean_amt_paid', 'median_amt_paid',
       'max_amt_paid', 'min_amt_paid', 'stddev_amt_paid'],
      dtype='object')

### Model train